In [ ]:
!uv sync
from copy import deepcopy

import numpy as np
from IPython.display import Image
import plotly.graph_objects as go

from scipy.optimize import fsolve

#### Elastic mill spring - extension from the former notebook

The mill stand is usually treated as rigid, which is not the reality. To be absolutely accurate, the mill stand should be treated as a system of springs. which includes the mill stand itself, the parts for the bearing of the rolls and the rolls themselves. Their reaction can be seen in the following diagram for defined mill. But the amplitude of this phenomenon is different from mill stand to mill stand and needs to be specified for each of those.

![](Auffederung.png)

The mill stand dependent spring can be simplified as a straight line with a positive slope, here as the function $mill \, stand \, linear$, and the material influence is approximated by a polynomial ($material \, poly$). This is an example and the function cannot be directly translated to a different mill stand.

In [ ]:
def mill_stand_linear(x, move_x=0):
    """Gerüstkennlinie"""
    return 704.1 * (x - move_x)


def x_values_material_poly(move_x=0):
    """Cut of for material characteristic curve, can be approximated as a vertical line from this point onwards"""
    return np.linspace(0.0, 3.0 + move_x, 300)


def material_poly(x, move_x=0):
    """Material characteristic curve"""
    return (
        -97.311 * ((x - move_x) ** 5)
        + 801.58 * ((x - move_x) ** 4)
        - 2569.9 * ((x - move_x) ** 3)
        + 4090.9 * ((x - move_x) ** 2)
        - 3681.1 * (x - move_x)
        + 2708.8
    )


def intercept(x, move_x_mill=0, move_x_material=0):
    """Function for finding the intercept point between both curves"""
    return mill_stand_linear(x, move_x=move_x_mill) - material_poly(
        x, move_x=move_x_material
    )

In this part, first the roll gap is set (in this case to 0.7) and the intercept point and the rolling force at that point is calculated. Afterwards, the plot shows the

In [ ]:
set_roll_gap = 0.7
# Calculation of intercept point and rolling force at that point
x_intercept = fsolve(intercept, x0=np.array([1.5]), args=(set_roll_gap,))
force_intercept = mill_stand_linear(x_intercept, move_x=set_roll_gap)

# Plot
fig = go.Figure()
x_values = np.linspace(0.0, 3.5, 300)
fig.add_trace(
    go.Scatter(
        x=x_values,
        y=mill_stand_linear(x_values, move_x=set_roll_gap),
        mode="lines",
        name="Gerüstkennlinie",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=np.append(x_values_material_poly(), [3.0]),
        y=np.append(material_poly(x_values_material_poly()), [0.0]),
        mode="lines",
        name="Materialkennlinie",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=np.append([0], x_intercept),
        y=np.append(force_intercept, force_intercept),
        mode="lines",
        name="voraussichtliche Walzkraft",
        line=dict(color="blue", dash="dash"),
    )
)
fig.add_trace(
    go.Scatter(
        x=np.append(x_intercept, x_intercept),
        y=np.append([0], force_intercept),
        mode="lines",
        name="voraussichtlicher Walzspalt",
        line=dict(color="blue", dash="dash"),
    )
)

fig.update_layout(
    title="Walzkraft vs Walzspalt / Walzgutdicke",
    xaxis_title="Walzspalt s / Walzgutdicke h",
    yaxis_title="Walzkraft F",
    xaxis=dict(range=[0, 4.0]),
    yaxis=dict(range=[0, 2000]),
    width=650,
    height=600,
)
print(
    f"The set roll gap is was {np.round(set_roll_gap, 2)} and the starting height of the work piece was {3.0}. In the end the real roll gap and therefore the end height of the work piece was {np.round(x_intercept, 2)}. "
)
fig.show()

Now we can also change the starting height of the material, which leads to a shift of the material dependent curve which therefore also leads to a change in the real roll gap and the end height of the workpiece.

In [ ]:
set_roll_gap = 0.7
change_starting_height = -0.2
x_intercept = fsolve(
    intercept, x0=np.array([1.5]), args=(set_roll_gap, change_starting_height)
)
force_intercept = mill_stand_linear(x_intercept, move_x=set_roll_gap)

fig_change = deepcopy(fig)
fig_change.add_trace(
    go.Scatter(
        x=x_values,
        y=mill_stand_linear(x_values, move_x=set_roll_gap),
        mode="lines",
        name="verschobene Gerüstkennlinie",
        line=dict(color="green"),
    )
)
fig_change.add_trace(
    go.Scatter(
        x=np.append(
            x_values_material_poly(change_starting_height),
            [3.0 + change_starting_height],
        ),
        y=np.append(
            material_poly(
                x_values_material_poly(change_starting_height),
                move_x=change_starting_height,
            ),
            [0],
        ),
        mode="lines",
        name="verschobene Materialkennlinie",
        line=dict(color="green"),
    )
)
fig_change.add_trace(
    go.Scatter(
        x=np.append([0], x_intercept),
        y=np.append(force_intercept, force_intercept),
        mode="lines",
        name="voraussichtliche Walzkraft",
        line=dict(color="green", dash="dash"),
    )
)
fig_change.add_trace(
    go.Scatter(
        x=np.append(x_intercept, x_intercept),
        y=np.append([0], force_intercept),
        mode="lines",
        name="voraussichtlicher Walzspalt",
        line=dict(color="green", dash="dash"),
    )
)
print(
    f"The set roll gap is was {np.round(set_roll_gap, 2)} and the starting height of the work piece was {3.0 + change_starting_height}. In the end the real roll gap and therefore the end height of the work piece was {np.round(x_intercept, 2)}. "
)
fig_change.show()

For this example case the system of springs is simplified as a single spring constant $C_S$. With the material dependent curve not known, but the mill stand dependent curve and therefore $C_S$ and the rolling force $F_W$ (for example through previous calculations) being available the real roll gap $s_1$ can be, according to Weber, calculated by:
$$s_1 = s_0 + {F_W / C_S}$$
Therefore if we import the results of the previous calculation, and use a known $C_S$ of one of our rolling mills as a base, we can calculate the real roll gap.

This runs the notebook from yesterday again and print the roll force and the set roll gap.

In [ ]:
%run heights_clad.ipynb

f

In [ ]:
cs = 805625
real_roll_gap = roll_gap * 1e3 + roll_force[0] / cs
print(
    f"Therefore, the set roll gap was {roll_gap * 1e3} mm and the real roll gap would be {np.round(real_roll_gap, 2)} mm. A difference of {np.round(real_roll_gap - roll_gap * 1e3, 2)} mm."
)